In [1]:
import mlflow
import pandas as pd
from gpt4all import GPT4All

# Connect to local MLflow server
mlflow.set_tracking_uri("http://127.0.0.1:5000")
experiment = mlflow.get_experiment_by_name("Fraud_Detection_Comparison_v1")
df_runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id]).head(9)

# Step 1: Create the first part of the prompt (metrics comparison only)
def create_comparison_prompt_part1(df, target_run_id):
    target_row = df[df["run_id"] == target_run_id].iloc[0]
    prompt = (
        f"Compare the following ML models to the target model (Run ID: {target_run_id}). "
        "Use accuracy, precision, and recall to analyze performance differences.\n\n"
    )

    for idx, row in df.iterrows():
        if row["run_id"] == target_run_id:
            continue
        model_type = row.get("params.model", "N/A")
        prompt += f"Model {idx + 1} ({model_type}):\n"
        prompt += f" - Run ID: {row['run_id']}\n"
        for metric in ["accuracy", "precision", "recall"]:
            col = f"metrics.{metric}"
            if col in row and pd.notnull(row[col]):
                prompt += f" - {metric.capitalize()}: {row[col]:.4f}\n"
        prompt += "\n"

    prompt += (
        f"--- TARGET MODEL ---\n"
        f"Target Model ({df[df['run_id'] == target_run_id].index[0] + 1}):\n"
        f" - Run ID: {target_run_id}\n"
        f" - Accuracy: {target_row['metrics.accuracy']:.4f}\n"
        f" - Precision: {target_row['metrics.precision']:.4f}\n"
        f" - Recall: {target_row['metrics.recall']:.4f}\n"
    )
    return prompt

# Step 2: Second prompt – ranking based on the first analysis
def create_followup_prompt():
    return (
        "\nNow based on the above comparison, rank all the models from best to worst.\n"
        "Include both the model type and Run ID in the ranking and explain why you chose this order."
    )

# Load the model
model = GPT4All("Llama-3.2-3B-Instruct-Q4_0.gguf")

# Get the target model
target_run_id = df_runs.iloc[0]["run_id"]

# === Part 1: Comparison only ===
prompt1 = create_comparison_prompt_part1(df_runs, target_run_id)
response1 = model.generate(prompt1, max_tokens=1024, temp=0.7)

# === Part 2: Ranking and explanation ===
prompt2 = response1 + create_followup_prompt()
response2 = model.generate(prompt2, max_tokens=1024, temp=0.7)

# Combine both responses
full_response = response1 + "\n\n" + response2

# Save the final output
with open("artifacts/ai_model_comparison.txt", "w", encoding="utf-8") as f:
    f.write(full_response)

print("AI Agent Response:\n")
print(full_response)


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/ai_model_comparison.txt'